In [17]:
import os
import json
import itertools
DIR = "../optimization/"
FILENAMES = {    
        "optipc": "02-opt-evaluation-ipc-coverage.json",
        "opt14": "03-opt-evaluation-new2014-coverage.json",
        "opt20": "04-opt-evaluation-new2020-coverage.json",
        "satipc": "05-sat-evaluation-ipc-coverage.json",
        "sat14": "06-sat-evaluation-new2014-coverage.json",
        "sat20": "07-sat-evaluation-new2020-coverage.json",
    }

DOMAINS = [
    "barman", "blocksworld", "childsnack", "data-network", "depot", "driverlog",
    "elevators", "floortile", "grid", "gripper", "hiking", "logistics",
    "miconic", "nomystery", "openstacks", "parking", "rovers", "satellite",
    "scanalyzer", "snake", "storage", "tpp", "transport", "visitall",
    "woodworking", "zenotravel",
]

PLANNERS = {
        "ipc2018-opt-complementary2-3584mb-1800s",
        "ipc2018-opt-decstar",
        "ipc2018-opt-delfi-blind",
        "ipc2018-opt-delfi-celmcut",
        "ipc2018-opt-delfi-ipdb-1800s",
        "ipc2018-opt-delfi-mas-miasm",
        "ipc2018-opt-delfi-mas-sccdfp-1800s",
        "ipc2018-opt-scorpion",
        "ipc2018-agl-decstar-agl",
        "ipc2018-agl-fd-remix",
        "ipc2018-agl-lapkt-bfws-pref",
        "ipc2018-agl-lapkt-dual-bfws",
        "ipc2018-agl-lapkt-poly-bfws",
        "ipc2018-agl-olcff",
        "ipc2018-agl-saarplan",

    }

comparisons = {}
COVERAGE = {}

for name, filename in FILENAMES.items():
    with open(os.path.join(DIR, "results", filename)) as f:
        data = json.load(f)
        
        for domain in DOMAINS: 
            coverage = data[domain]
            COVERAGE[(name, domain)]= coverage
            comparisons[(name, domain)] = sum ([1 for (p1, p2) in itertools.combinations(coverage.keys(), 2) if p1 in PLANNERS and p2 in PLANNERS and coverage[p1] != coverage[p2]])
            
    # Only keep the data for a subset of planners and domains.
    #filtered_dict = {}
    #for domain in DOMAINS:
    #    domain_dict = {planner: coverage for planner, coverage in d[domain].items() if planner in PLANNERS}
    #    filtered_dict[domain] = domain_dict
    #dicts[name] = filtered_dict

#algo_dicts = []
#for domain_dict in dicts.values():
#    algo_dicts.extend(domain_dict.values())
    
#dicts



In [12]:
LOGDIR = "../logfiles/2020-09-03/"
import os
import re
import pandas as pd

from IPython.display import display

#2020-11-17 22:58:38,135 INFO     Different evaluated sequences: 202
#2020-11-17 22:58:38,222 INFO     Candidate sequences: 100
#2020-11-17 22:58:38,480 INFO     Solution value  = 0.0

ATRIBUTES_TABLE = ["track", "domain", "evaluated_sequences", "candidate_sequences", "solution_value", "sart_runtimes", "num_sequences", "min_penalty", "max_penalty", "comparisons", "comparisonsipc"]

regex_int = [re.compile(x) for x in [".*Different evaluated sequences: (?P<evaluated_sequences>(\d+))",
                                     ".*Candidate sequences: (?P<candidate_sequences>(\d+))", 
                                    ]]
regex_float = [re.compile(x) for x in [".*Solution value  = (?P<solution_value>(.*))",]]
DATA_LOGS = []
table = []

for logfile in sorted(os.listdir(LOGDIR)):
    _, track, domain = logfile.split("_")
    f = open(f"{LOGDIR}/{logfile}")
    content = f.readlines()
    penalties = [float(l.split(":")[-1]) for l in content if l.startswith("Penalty: ")]
    
    data = { "track" : track, "domain" : domain, "sart_runtimes" : False, "num_sequences" : 0, "min_penalty" : min(penalties) if penalties else "-", 
            "max_penalty" : max(penalties) if penalties else "-", 
            "comparisons" : comparisons[(track, domain.replace('depots','depot').replace('rover','rovers').replace('-strips',''))], 
            "comparisonsipc" : comparisons[(track, domain.replace('depots','depot').replace('rover','rovers').replace('-strips',''))] - comparisons[(track.replace('14','ipc').replace('20','ipc'), domain.replace('depots','depot').replace('rover','rovers').replace('-strips',''))]}
            
    for l in content: 
        if "Using sart runtimes on CPLEX optimization" in l: 
            data["sart_runtimes"] = True
            continue
        if "Selected: sequence" in l:
            data["num_sequences"] += 1

            
        for reg in regex_int: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = int(value)
        
        for reg in regex_float: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = float(value)

    if data["comparisonsipc"]>=0: 
        continue
        
    table.append([data[atr] for atr in ATRIBUTES_TABLE])

    DATA_LOGS.append(data)
    
#print("\n".join([f'{d["track"]} {d["domain"]}' for d in DATA_LOGS if not d["sart_runtimes"]]))
pd.options.display.float_format = "{:,.2g}".format
pd.set_option('display.max_rows', 500)

df = pd.DataFrame(table,columns =  ATRIBUTES_TABLE)
display(df)


                 

,track,domain,evaluated_sequences,candidate_sequences,solution_value,sart_runtimes,num_sequences,min_penalty,max_penalty,comparisons,comparisonsipc
0,opt14,data-network,251,180,18,True,2,-,-,25,-2
1,opt14,hiking,438,56,0,True,3,-,-,15,-8
2,opt14,openstacks,722,390,0,True,2,-,-,18,-6
3,opt14,parking,30,4,71,True,3,-,-,24,-4
4,opt14,rover,843,100,12,True,3,-,-,22,-3
5,opt14,scanalyzer,18,7,2.5e+02,True,2,-,-,24,-3
6,opt14,transport,139,99,0,True,2,-,-,18,-3
7,opt20,depots,322,100,16,True,3,-,-,24,-2
8,opt20,elevators,252,80,17,True,3,-,-,23,-3
9,opt20,logistics,97,91,16,True,1,-,-,25,-2


In [22]:
for data in DATA_LOGS: 
    if data['comparisonsipc'] < -10:
        track, domain = data['track'], data['domain']
        
        print (track, domain)
        print(COVERAGE[(track, domain.replace('rover','rovers').replace('depots','depot'))])
        print(COVERAGE[(track.replace('14','ipc').replace('20','ipc'), domain.replace('rover','rovers').replace('depots','depot'))])


            

opt20 openstacks
{'fd1906-bjolp': 3, 'fd1906-blind': 3, 'fd1906-fdss1-mas1': 1, 'fd1906-fdss1-mas2': 3, 'fd1906-lmcut': 3, 'ipc2014-opt-symba1': 18, 'ipc2018-opt-complementary2-3584mb-1800s': 7, 'ipc2018-opt-decstar': 3, 'ipc2018-opt-decstar-fork': 0, 'ipc2018-opt-decstar-star': 3, 'ipc2018-opt-delfi-blind': 3, 'ipc2018-opt-delfi-celmcut': 3, 'ipc2018-opt-delfi-ipdb-1800s': 3, 'ipc2018-opt-delfi-mas-miasm': 3, 'ipc2018-opt-delfi-mas-sccdfp-1800s': 3, 'ipc2018-opt-scorpion': 3}
{'fd1906-bjolp': 40, 'fd1906-blind': 71, 'fd1906-fdss1-mas1': 31, 'fd1906-fdss1-mas2': 42, 'fd1906-lmcut': 42, 'ipc2014-opt-symba1': 70, 'ipc2018-opt-complementary2-3584mb-1800s': 64, 'ipc2018-opt-decstar': 71, 'ipc2018-opt-decstar-fork': 0, 'ipc2018-opt-decstar-star': 40, 'ipc2018-opt-delfi-blind': 47, 'ipc2018-opt-delfi-celmcut': 44, 'ipc2018-opt-delfi-ipdb-1800s': 47, 'ipc2018-opt-delfi-mas-miasm': 44, 'ipc2018-opt-delfi-mas-sccdfp-1800s': 47, 'ipc2018-opt-scorpion': 42}
opt20 rover
{'fd1906-bjolp': 6, 'fd1906